In [1]:
import os
import torch
import matplotlib.pyplot as plt

def show(tensor_path):
    steps = 50  # 假设有50个step
    channels = 16  # 假设有16个channel
    t_group_size = 3  # 每 3 个 T 进行分组
    
    # 存储最终的均值和标准差
    mean_values = torch.zeros(steps)
    std_values = torch.zeros(steps)
    
    for i in range(steps):
        # 加载 noise_pred 数据
        noise_pred = torch.load(f"/home/siyuan/HunyuanVideo_efficiency/results/{tensor_path}/noise_pred_{i}.pt")[0, :, :, :, :]
        
        batch_size, num_channels, num_timesteps, height, width = noise_pred.shape
        
        num_t_groups = (num_timesteps + t_group_size - 1) // t_group_size  # 计算 T 需要多少个分组
        
        group_means = []
        group_stds = []
        
        # 遍历 T 维度，每 3 个 T 作为一组
        for t_group in range(num_t_groups):
            t_start = t_group * t_group_size
            t_end = min((t_group + 1) * t_group_size, num_timesteps)  # 确保不会超出 T 维度
            
            # 计算每个channel的均值和标准差
            group_mean = noise_pred[:, :, t_start:t_end, :, :].mean(dim=[2, 3, 4])  # (B, C)
            group_std = noise_pred[:, :, t_start:t_end, :, :].std(dim=[2, 3, 4])  # (B, C)
            
            group_means.append(group_mean)
            group_stds.append(group_std)
        
        # 转换为张量并计算所有 channel 的均值
        group_means = torch.stack(group_means).mean()  # 计算所有 channel 均值的均值
        group_stds = torch.stack(group_stds).mean()    # 计算所有 channel 标准差的均值
        
        mean_values[i] = group_means
        std_values[i] = group_stds
    
    # 绘制最终图表
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 绘制均值变化
    axes[0].plot(range(steps), mean_values, marker='x', linestyle='-', label='Mean over all Channels')
    axes[0].set_title('Mean of Noise Prediction')
    axes[0].set_xlabel('Step (i)')
    axes[0].set_ylabel('Mean')
    
    # 绘制标准差变化
    axes[1].plot(range(steps), std_values, marker='x', linestyle='-', label='Standard Deviation over all Channels')
    axes[1].set_title('Standard Deviation of Noise Prediction')
    axes[1].set_xlabel('Step (i)')
    axes[1].set_ylabel('Standard Deviation')
    
    plt.show()


In [3]:
show("tensor")

/tmp/ipykernel_121419/139410939.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  noise_pred = torch.load(f"/home/siyuan/HunyuanVideo_efficiency/results/{tensor_path}/noi

ValueError: not enough values to unpack (expected 5, got 4)